# Ozone mixing ratio

## Import libraries

In [ ]:
import os

import cdsapi
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
from c3s_eqc_automatic_quality_control import download

plt.style.use("seaborn-v0_8-notebook")

## Set parameters

In [ ]:
os.environ["CDSAPI_RC"] = os.path.expanduser("~/calmanti_sandro/.cdsapirc")

## Set requests

In [ ]:
collection_id = "satellite-ozone-v1"
requests = {
    "limb": {
        "processing_level": "level_3",
        "variable": "mole_concentration_of_ozone_in_air",
        "vertical_aggregation": "vertical_profiles_from_limb_sensors",
    },
    "nadir": {
        "processing_level": "level_3",
        "variable": "mole_content_of_ozone_in_atmosphere_layer",
        "vertical_aggregation": "vertical_profiles_from_nadir_sensors",
    },
}

## Functions to cache

In [ ]:
def preprocess(ds):
    if "time" in ds.dims:
        return ds
    time_str = ds.attrs["time_coverage_start"][:6]
    time = pd.to_datetime(time_str, format="%Y%m")
    return ds.expand_dims(time=[time])


def compute_ozone_mixing_ratio(ds, rconst=8.314):
    indexers = {"latitude": slice(50, 70)}
    if "altitude" in ds.dims:
        indexers["altitude"] = 50_000
    else:
        indexers["pressure"] = 50
    ds = ds.sel(**indexers, drop=True)

    ds = ds.mean(set(ds.dims) - {"time", "latitude"}, keep_attrs=True)
    if (coarsen := ds.sizes["latitude"] // 2) > 1:
        ds = ds.coarsen(latitude=coarsen).mean(keep_attrs=True)

    if "ozone_mixing_ratio" in ds:
        da = ds["ozone_mixing_ratio"]
    else:
        da = ds["ozone_concentration"]
        da = ((da * rconst * ds["temperature"]) / ds["pressure"]) * 1e4
        da.attrs = {"long_name": "O3 mixing ratio", "units": "ppm"}
    return da.to_dataset(name="ozone_mixing_ratio")


def download_combined_dataset(collection_id, request):
    collection = cdsapi.Client(quiet=True).client.get_collection(collection_id)

    datasets = []
    for sensor in collection.apply_constraints(request)["sensor"]:
        if sensor in ["cllg", "cmzm", "merged_np"]:
            continue  # no merged products
        sensor_r = request | {"sensor": sensor}
        for algo in collection.apply_constraints(sensor_r)["algorithm"] or [None]:
            algo_r = sensor_r | {"algorithm": algo or []}
            for version in collection.apply_constraints(algo_r)["version"]:
                version_r = algo_r | {"version": version}
                requests = []
                for year in collection.apply_constraints(version_r)["year"]:
                    year_r = version_r | {"year": year}
                    months = collection.apply_constraints(year_r)["month"]
                    requests.append(year_r | {"month": months})

                product = "-".join([sensor, version] + ([algo] if algo else []))
                if product == "omps-v0002-usask":
                    continue  # time_coverage_start attribute is missing
                print(f"{product=}")

                ds = download.download_and_transform(
                    collection_id,
                    requests,
                    preprocess=preprocess,
                    transform_func=compute_ozone_mixing_ratio,
                )
                datasets.append(ds.expand_dims(product=[product]))
    return xr.concat(datasets, "product")

## Download and tranform

In [ ]:
datasets = {}
for sensor, request in requests.items():
    print(f"{sensor=}")
    datasets[sensor] = download_combined_dataset(collection_id, request)

## Quick and dirty plots

In [ ]:
for sensor, ds in datasets.items():
    (da,) = ds.data_vars.values()
    da = da.dropna("product", how="all")
    cmap = plt.get_cmap("viridis", da.sizes["product"])
    colors = [cmap(i) for i in range(da.sizes["product"])]
    with plt.rc_context({"axes.prop_cycle": plt.cycler(color=colors)}):
        facet = da.plot(row="latitude", hue="product", figsize=(10, 6))
    for ax in facet.axs.flatten():
        ax.grid()
    plt.show()